In [4]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('sqlite_db_pythonsqlite.db')

# Helper function to run queries
def run_query(query):
    df = pd.read_sql(query, conn)
    display(df)

In [8]:
q1 = """
SELECT name
FROM Facilities
WHERE membercost > 0;
"""
run_query(q1)

,name
0,Tennis Court 1
1,Tennis Court 2
2,Massage Room 1
3,Massage Room 2
4,Squash Court


In [10]:
q2 = """
SELECT COUNT(*) AS free_facilities
FROM Facilities
WHERE membercost = 0;
"""
run_query(q2)

,free_facilities
0,4


In [12]:
q3 = """
SELECT facid, name, membercost, monthlymaintenance
FROM Facilities
WHERE membercost > 0 
  AND membercost < (0.2 * monthlymaintenance);
"""
run_query(q3)

,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,4,Massage Room 1,9.9,3000
3,5,Massage Room 2,9.9,3000
4,6,Squash Court,3.5,80


In [14]:
q4 = """
SELECT *
FROM Facilities
WHERE facid IN (1,5);
"""
run_query(q4)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


In [16]:
q5 = """
SELECT name, monthlymaintenance,
       CASE WHEN monthlymaintenance > 100 
            THEN 'expensive' 
            ELSE 'cheap' END AS cost_label
FROM Facilities;
"""
run_query(q5)

,name,monthlymaintenance,cost_label
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive
5,Massage Room 2,3000,expensive
6,Squash Court,80,cheap
7,Snooker Table,15,cheap
8,Pool Table,15,cheap


In [18]:
q6 = """
SELECT firstname, surname, joindate
FROM Members
WHERE joindate = (SELECT MAX(joindate) FROM Members);
"""
run_query(q6)


,firstname,surname,joindate
0,Darren,Smith,2012-09-26 18:08:45


In [22]:
q7 = """
SELECT DISTINCT f.name AS court,
       m.firstname || ' ' || m.surname AS member_name
FROM Bookings b
JOIN Facilities f ON b.facid = f.facid
JOIN Members m ON b.memid = m.memid
WHERE f.name LIKE 'Tennis Court%'
ORDER BY member_name;
"""
run_query(q7)


,court,member_name
0,Tennis Court 1,Anne Baker
1,Tennis Court 2,Anne Baker
2,Tennis Court 2,Burton Tracy
3,Tennis Court 1,Burton Tracy
4,Tennis Court 1,Charles Owen
5,Tennis Court 2,Charles Owen
6,Tennis Court 2,Darren Smith
7,Tennis Court 1,David Farrell
8,Tennis Court 2,David Farrell
9,Tennis Court 2,David Jones


In [24]:
q8 = """
SELECT f.name AS facility,
       m.firstname || ' ' || m.surname AS member_name,
       CASE WHEN b.memid = 0 THEN f.guestcost*b.slots 
            ELSE f.membercost*b.slots END AS cost
FROM Bookings b
JOIN Facilities f ON b.facid = f.facid
JOIN Members m ON b.memid = m.memid
WHERE date(b.starttime) = '2012-09-14'
  AND (CASE WHEN b.memid = 0 THEN f.guestcost*b.slots 
            ELSE f.membercost*b.slots END) > 30
ORDER BY cost DESC;
"""
run_query(q8)

,facility,member_name,cost
0,Massage Room 2,GUEST GUEST,320.0
1,Massage Room 1,GUEST GUEST,160.0
2,Massage Room 1,GUEST GUEST,160.0
3,Massage Room 1,GUEST GUEST,160.0
4,Tennis Court 2,GUEST GUEST,150.0
5,Tennis Court 1,GUEST GUEST,75.0
6,Tennis Court 1,GUEST GUEST,75.0
7,Tennis Court 2,GUEST GUEST,75.0
8,Squash Court,GUEST GUEST,70.0
9,Massage Room 1,Jemima Farrell,39.6


In [26]:
q9 = """
SELECT facility, member_name, cost
FROM (
    SELECT f.name AS facility,
           m.firstname || ' ' || m.surname AS member_name,
           CASE WHEN b.memid = 0 THEN f.guestcost*b.slots 
                ELSE f.membercost*b.slots END AS cost
    FROM Bookings b
    JOIN Facilities f ON b.facid = f.facid
    JOIN Members m ON b.memid = m.memid
    WHERE date(b.starttime) = '2012-09-14'
) sub
WHERE cost > 30
ORDER BY cost DESC;
"""
run_query(q9)

,facility,member_name,cost
0,Massage Room 2,GUEST GUEST,320.0
1,Massage Room 1,GUEST GUEST,160.0
2,Massage Room 1,GUEST GUEST,160.0
3,Massage Room 1,GUEST GUEST,160.0
4,Tennis Court 2,GUEST GUEST,150.0
5,Tennis Court 1,GUEST GUEST,75.0
6,Tennis Court 1,GUEST GUEST,75.0
7,Tennis Court 2,GUEST GUEST,75.0
8,Squash Court,GUEST GUEST,70.0
9,Massage Room 1,Jemima Farrell,39.6


In [28]:
q10 = """
SELECT f.name,
       SUM(CASE WHEN b.memid = 0 THEN f.guestcost*b.slots 
                ELSE f.membercost*b.slots END) AS revenue
FROM Bookings b
JOIN Facilities f ON b.facid = f.facid
GROUP BY f.name
HAVING revenue < 1000
ORDER BY revenue;
"""
run_query(q10)

,name,revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


In [30]:
q11 = """
SELECT m.firstname || ' ' || m.surname AS member,
       r.firstname || ' ' || r.surname AS recommender
FROM Members m
LEFT JOIN Members r ON m.recommendedby = r.memid
ORDER BY m.surname, m.firstname;
"""
run_query(q11)

,member,recommender
0,Florence Bader,Ponder Stibbons
1,Anne Baker,Ponder Stibbons
2,Timothy Baker,Jemima Farrell
3,Tim Boothe,Tim Rownam
4,Gerald Butters,Darren Smith
5,Joan Coplin,Timothy Baker
6,Erica Crumpet,Tracy Smith
7,Nancy Dare,Janice Joplette
8,David Farrell,None
9,Jemima Farrell,None


In [32]:
q12 = """
SELECT f.name, COUNT(*) AS usage_count
FROM Bookings b
JOIN Facilities f ON b.facid = f.facid
WHERE b.memid != 0
GROUP BY f.name;
"""
run_query(q12)

,name,usage_count
0,Badminton Court,344
1,Massage Room 1,421
2,Massage Room 2,27
3,Pool Table,783
4,Snooker Table,421
5,Squash Court,195
6,Table Tennis,385
7,Tennis Court 1,308
8,Tennis Court 2,276


In [34]:
q13 = """
SELECT f.name,
       strftime('%Y-%m', b.starttime) AS month,
       COUNT(*) AS usage_count
FROM Bookings b
JOIN Facilities f ON b.facid = f.facid
WHERE b.memid != 0
GROUP BY f.name, month
ORDER BY month, f.name;
"""
run_query(q13)

,name,month,usage_count
0,Badminton Court,2012-07,51
1,Massage Room 1,2012-07,77
2,Massage Room 2,2012-07,4
3,Pool Table,2012-07,103
4,Snooker Table,2012-07,68
5,Squash Court,2012-07,23
6,Table Tennis,2012-07,48
7,Tennis Court 1,2012-07,65
8,Tennis Court 2,2012-07,41
9,Badminton Court,2012-08,132
